In [97]:
import requests
requests.packages.urllib3.disable_warnings() 
import json
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px

###### Input your access_token here!######
access_token = 'xxxxxxxxxxxxxxx'

# Import the time-series representing your historical demand
infile = 'https://raw.githubusercontent.com/alvin-chan/bdt_django_front_end_postgres_mkdocs/master/example_data/bdt_example_input.csv'
input_df = pd.read_csv(infile)
# Convert to a payload for WasteNot, adding in parameters for:
# - Shelf life of 2 days
# - Costs $1/unit, Revenue generated $5/unit sold
# - Optimise for maximum profit (revenue less cost)
# - Forecast 7 days ahead after the end of the historical time-series

input_dict = {'timestamp':list(input_df['timestamp'].values),
              'demand':[int(x) for x in input_df['demand_units'].values],
              "cost": 1, "sale_price": 5, 
              "shelf_life_seconds": 172800.0, 
              "opt_param": "profit",
              "delivery_dayofweek": [0,1,2,3,4,5,6],
              "forecast_periods_ahead":7,
              }

r = json.dumps(input_dict)
loaded_r = json.loads(r)

In [98]:
# Calling the WasteNot API service with the payload
url = "https://api.bluedotthinking.com/forecast"
payload=r

headers = {
    'Content-Type': 'application/json',
    'access_token': access_token
}

response = requests.request("POST", url, headers=headers, data=payload, verify=True)
print (response)

# Loading the response into JSON format
json_data = json.loads(response.text)

# Load the output payload into pandas dataframes
delivery_df = pd.DataFrame(data={'timestamp':json_data['delivery_timestamp'],
                                 'delivered_units':json_data['delivery_units']
                                })

forecast_df = pd.DataFrame(data={'timestamp': json_data['forecast_timestamp'],
                                 'predicted_demand_units': json_data['predicted_demand_units'],
                                 'optimised_demand_units': json_data['optimised_demand_units'],
                                })

<Response [200]>


In [99]:
# Plotting:
# a) Historical demand for the last 90 days, 
# b) Units to be delivered using the optimised forecast over the next 7 days

fig = go.Figure()
fig.update_layout(title="Historical & Forecast Demand using WasteNot")
fig.add_trace(
    go.Scatter(x=delivery_df['timestamp'].values, y=delivery_df['delivered_units'].values,
           name="Forecast Units to Deliver", line_color='blue'))
fig.add_trace(
    go.Scatter(x=input_df['timestamp'].values[-90:], y=input_df['demand_units'].values[-90:],
          name="Historical Demand Units", line_color='black'))
fig.show()

In [100]:
# Visualising the tables containing the delivered units (optimised), by day
delivery_df

,timestamp,delivered_units
0,2018-08-11,1581.0
1,2018-08-12,1272.0
2,2018-08-13,1489.0
3,2018-08-14,1439.0
4,2018-08-15,1543.0
5,2018-08-16,1678.0
6,2018-08-17,2262.0


In [101]:
# Visualising the tables containing the forecasted demand units (unoptimised and optimised), by day
forecast_df

,timestamp,predicted_demand_units,optimised_demand_units
0,2018-08-11,1483.0,1581.0
1,2018-08-12,1193.0,1272.0
2,2018-08-13,1397.0,1489.0
3,2018-08-14,1350.0,1439.0
4,2018-08-15,1448.0,1543.0
5,2018-08-16,1574.0,1678.0
6,2018-08-17,2122.0,2262.0
